<a href="https://colab.research.google.com/github/ucfilho/raianars_adjust_RTC/blob/master/RTC_adjust_BugBuster_v3_mar_08_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x):
  global Trocas,D, tempInicial, tempAtual 
  cols=len(tempInicial) 
  Ind_I=np.zeros((D,Trocas)).astype(np.int64)
  Ind_J=np.zeros((D,Trocas)).astype(np.int64)
  Tin=np.zeros((D,Trocas))
  Tout=np.zeros((D,Trocas))
  deltaT=np.zeros((D,Trocas))
  for i in range(D):
    for j in range(Trocas):
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas] 
    for jj in range(cols):
      Tin[i,jj]=tempInicial[jj]
      Tout[i,jj]=tempAtual[jj]
  return Ind_I,Ind_J,deltaT, Tin, Tout

In [0]:
#def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):
def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout):
#def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,tempInicial,tempAtual):
  global Trocas,D
  # cols=len(mCp)
  # i represent hot stream  and j represent cold stream 
  # k represent sequential heat exchanges
  cargaTermica=np.zeros((D,Trocas))
  contador=0
  cont_uso=0
  erro=np.zeros((D)) # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for ki in range(D):
    for kj in range(Trocas):
      k=kj
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      cargaTermica[ki,kj]=mCp[i]*deltaT[ki,kj]
      tempSaidaQ=Tin[ki,i]+deltaT[ki,kj]
      tempSaidaF=-cargaTermica[ki,kj]/(mCp[j])+Tout[ki,j]
      flag=0 # flag : indica que nao entra no teste logico
      # usei flag no lugar q ifs pq a identacao ficou melhor assim
      if(Tin[ki,i] > Tin[ki,j]): #Tquente maior Tfria: vaores atuais
        if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
          # if(contador < cols):
          flag=1
            
      if(flag>0):
        Tout[ki,i]=tempSaidaQ
        Tout[ki,j]=tempSaidaF
      else:
        erro[ki]=1
        #erro[ki]=0


  return Tout,cargaTermica,erro

In [0]:
def Area(U,Tin,Tout,Ind_I,Ind_J,deltaT):

  global Trocas,mCp,D
  
  Area=[]
  print('Tout');print(Tout.shape)

  for ki in range(D):
    Area_Particula=[]
    for kj in range(Trocas):
      i=Ind_I[ki,kj]
      Carga=mCp[i]*deltaT[ki,kj]
      tempSaidaF=Tout[ki,i]
      tempSaidaQ=Tout[ki,i]
      deltaT1=Tin[ki,i]-tempSaidaF
      deltaT2=tempSaidaQ-Tin[ki,i]
      try:
        A=(deltaT1-deltaT2)
        B=np.log((deltaT1/deltaT2))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
  
      if(deltaTMediaLog==1e99):
        AreaCalc=1e99
      else:
        AreaCalc=abs(Carga/(U*deltaTMediaLog))
      
      #print(Tout)
      #print(Tin)
      #print("Tout[ki,i]==Tin[ki,i] %.2f %.2f"%(Tout[ki,i],Tin[ki,i]))
      #print("Tout[ki,j]==Tin[ki,j] %.2f %.2f"%(Tout[ki,j],Tin[ki,j]))
      
      if(Tout[ki,i]==Tin[ki,i]):
            AreaCalc=0

      if(Tout[ki,i]==Tin[ki,i]):
        AreaCalc=0
      Area_Particula.append(AreaCalc)
      
    Area.append(Area_Particula)

  return Area

In [0]:
def Utilidades(tempAlvo,Tout,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353

  
  Areas=[]
  cargaUQtodas=[]
  cargaUFtodas=[]

  for ki in range(D):
    areaTrocador=[]

    for j in range(cols):
      delta=tempAlvo[j]-Tout[ki,j]
      if(delta>0):#usa UQ
        cargaUQ=cargaUQ+mCp[j]*delta
        deltaT1=(TUQin-tempAlvo[j])
        deltaT2=(TUQout-Tout[ki,j])

        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99

        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

      elif(delta==0):#nao usa UF e UQ
        deltaTMediaLog=0
        areaTrocador.append(0) 

      else:#usa UF
        deltaT1=(Tout[ki,j]-TUFout)
        deltaT2=(tempAlvo[j]-TUFin)     
        cargaUF=cargaUF+mCp[j]*(-delta)

        #UF -> água resfriada -> entra a 303K e sai a 353K
        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99
        
        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))



    cargaUQtodas.append(cargaUQ)
    cargaUFtodas.append(cargaUF)

    Areas.append(areaTrocador)
  
  # print(Areas)
  #print('----carga todas-------')
  #print(cargaUQtodas)
  #print(cargaUFtodas)
  return Areas, cargaUQtodas,cargaUQtodas 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
def CustoProcesso(Area,Utilidades,cargaUFtodas,cargaUQtodas):
  global coefCustoUQ,coefCustoUF,D

  cost=np.zeros((D))
  print('Area=it===')
  print(Area)
  #print('cargaUFtodas-custo rotina')
  #print(cargaUFtodas)
  #print('cargaUQtodas-custo rotina')
  #print(cargaUQtodas)
  for i in range(D):
    cargaUQ=cargaUFtodas[i]
    cargaUF=cargaUFtodas[i]
    areaTrocadores=Area[i]
    areaUtilidades=Utilidades[i]
    costTrocadores=0
    for it in areaTrocadores:
      for wii in areaUtilidades:
        print('wii=%.3f it=%.3f'%(wii,it))
        if(it<1e99):
          if(wii<1e99):
            costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
            print('costTrocadores=',costTrocadores)
          else:
            costTrocadores=1e99
            break
          
    costUQ=coefCustoUQ*cargaUQ
    costUF=coefCustoUF*cargaUF
    print('costUQ %.2f costUF %.2f costTrocadores %.2f'%(costUQ,costUF,costTrocadores))
    if(costTrocadores==1e99):
      cost[i]=1e99
    else:
      cost[i]=costUQ+costUF+costTrocadores
      
  return cost

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  global D
  
  funr=np.zeros((D))
  cols=len(mCp)
  Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  Tout,Carga,Erro=Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout)
  areaTrocadores =Area(U,Tin,Tout,Ind_I,Ind_J,deltaT)
  areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,Tout,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr=cost


  for k in range(D):
    if(Erro[k]>0):
      funr[k]=1e99

  funr = np.where(np.isnan(cost), 1e99, cost)

  return funr,Erro

In [0]:
#********************PROGRAMA PRINCIPAL*************************
D=25
Trocas=4
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
tempAtual=np.copy(tempInicial) 
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]

coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]





In [0]:
cols=12
x=np.zeros((D,cols))
#MAX=np.array([3,2,3,2,0,1,1,1,0,0,0,0])
#MIN=np.array([3,2,3,2,0,1,1,1,-100,-100,-100,-100])
MAX=np.array([3,3,3,3,1,1,1,1,0,0,0,0])
#MIN=np.array([2,2,2,2,0,0,0,0,-100,-100,-100,-100])
MIN=np.array([2,2,2,2,0,0,0,0,-20,-20,-20,-20])
for i in range(D):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

In [0]:
#print(x)
#print(x.shape)

In [0]:
# First step: it is working fine...
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca



In [0]:
# second step
Tout,Carga,Erro=Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout)

In [14]:
print(Erro)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


In [15]:
# third step: it is not working????
areaTrocadores =Area(U,Tin,Tout,Ind_I,Ind_J,deltaT)


Tout
(25, 4)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log


In [16]:
# fourth step: it is working
areaTrocadores =Area(U,Tin,Tout,Ind_I,Ind_J,deltaT)


Tout
(25, 4)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log


In [17]:
# fiveth step
areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,Tout,mCp)
areaTrocadores =Area(U,Tin,Tout,Ind_I,Ind_J,deltaT)
areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,Tout,mCp)
cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)

Tout
(25, 4)
Area=it===
[[nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan]]
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
costUQ 39938.08 costUF 4429.50 costTrocadores 0.00
wii=16.662 it=nan
wii=10.684 it=nan
wii=6.395 it=nan
wii=8.422 it=nan
wii=16.662 it=nan
wii=10

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log


In [18]:
Custo, Erro=FUN(x)
''''''
print(Custo)
print(Erro)


Tout
(25, 4)
Area=it===
[[nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan]]
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
wii=14.724 it=nan
wii=11.421 it=nan
wii=6.632 it=nan
wii=8.318 it=nan
costUQ 39938.08 costUF 4429.50 costTrocadores 0.00
wii=16.662 it=nan
wii=10.684 it=nan
wii=6.395 it=nan
wii=8.422 it=nan
wii=16.662 it=nan
wii=10

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log


In [19]:
'''
[1179863.51077034 1181562.29317795 1180331.49642757 1181460.67704922
 1181095.7955166  1180257.42122825 1180111.61408429 1182266.47563206
 1180710.37174756 1182890.62261093 1181817.56790337 1180547.66398535
 1182338.81611602 1182913.35693227 1180353.18466131 1180648.20461183
 1179862.73888151 1181911.64223794 1179179.79482066 1180910.83199421
 1180822.39103782 1180981.81021803 1180531.21335749 1180458.18878147
 1182030.28999255]
'''

'\n[1179863.51077034 1181562.29317795 1180331.49642757 1181460.67704922\n 1181095.7955166  1180257.42122825 1180111.61408429 1182266.47563206\n 1180710.37174756 1182890.62261093 1181817.56790337 1180547.66398535\n 1182338.81611602 1182913.35693227 1180353.18466131 1180648.20461183\n 1179862.73888151 1181911.64223794 1179179.79482066 1180910.83199421\n 1180822.39103782 1180981.81021803 1180531.21335749 1180458.18878147\n 1182030.28999255]\n'

In [20]:
print(tempAtual)

[453 393 523 533]


In [21]:
# caso I: parecido, teste 
#Ind_x=[4,3,4,3]
#Ind_y=[1,2,2,2]
#deltaT=[-21,-75,-21,-27]
D=1
#xtest=np.array([3,2,3,2,0,1,1,1,-21,-75,-21,-27]).reshape(1,12)
xtest=np.array([3,2,3,2,0,1,1,1,-78,-75,-21,-27]).reshape(1,12)
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x)
print(Ind_I)
print(Ind_J)
Custo, Erro=FUN(xtest)
print(Custo)
print(Erro)

[[2 2 3 2]]
[[1 1 0 0]]
Tout
(1, 4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log



Area=it===
[[nan, nan, nan, nan]]
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
costUQ 40315.00 costUF 4471.30 costTrocadores 0.00
[1.e+99]
[1.]


In [22]:
#xtest=np.array([3,2,3,2,0,1,1,1,-75,-21,-27,-78]).reshape(1,12)
xtest=np.array([3,2,3,2,0,1,1,1,-78,-27,-21,-75]).reshape(1,12)
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x)
print(Ind_I)
print(Ind_J)
Custo, Erro=FUN(xtest)
print(Custo)
print(Erro)

[[2 2 3 2]]
[[1 1 0 0]]
Tout
(1, 4)
Area=it===
[[nan, nan, nan, nan]]
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
wii=18.432 it=nan
wii=8.825 it=nan
wii=5.802 it=nan
wii=8.228 it=nan
costUQ 40315.00 costUF 4471.30 costTrocadores 0.00
[1.e+99]
[1.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log
